In [1]:
import pandas as pd

In [ ]:
btc_df = pd.read_csv(r"C:\Users\David gathara marigi\Downloads\bitcoin_price.csv")

In [ ]:
btc_df['time'] = pd.to_datetime(btc_df['time'], unit='ms') 
btc_df.set_index('time', inplace=True) 
btc_df = btc_df[['close', 'volume']]

In [ ]:
btc_df['close_1d_future'] = btc_df['close'].shift(-1) 
btc_df.dropna(inplace=True)

In [ ]:
features = btc_df.drop('close_1d_future', axis=1) 
targets = btc_df['close_1d_future'] 
train_idx = int(0.75 * btc_df.shape[0]) 
x_train = features.iloc[:train_idx] 
y_train = targets.iloc[:train_idx] 
x_test = features.iloc[train_idx:] 
y_test = targets.iloc[train_idx:]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
l1_lr = LassoCV(cv=TimeSeriesSplit()) 
l1_lr.fit(x_train, y_train) 
print(mae(y_train, l1_lr.predict(x_train))) 
print(mae(y_test, l1_lr.predict(x_test)))

In [ ]:
train_dates = btc_df.index[:train_idx] 
test_dates = btc_df.index[train_idx:] 
train_predictions = l1_lr.predict(x_train) 
test_predictions = l1_lr.predict(x_test) 
plt.plot_date(train_dates, y_train, fmt='-', color='b') 
plt.plot_date(train_dates, train_predictions, fmt='--', 
color='orange') 
plt.plot_date(test_dates, y_test, fmt='-', color='k') 
plt.plot_date(test_dates, test_predictions, fmt='--', 
color='r') 
plt.axvline(btc_df.index[train_idx]) 
plt.yscale('log')

In [ ]:
btc_df.reset_index(inplace=True) 
btc_df.drop('close_1d_future', axis=1, inplace=True) 
btc_df.rename(columns={'close': 'y', 'time': 'ds'}, 
inplace=True)

In [ ]:
from prophet import Prophet

In [ ]:
m = Prophet() 
m.fit(btc_df)

In [ ]:
future = m.make_future_dataframe(periods=365) 
forecast = m.predict(future) 
m.plot(forecast)

In [ ]:
m.plot_components(forecast)